In [6]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import wave
import scipy.signal as sg
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")
plt.ion()

In [24]:
INF = 100

def calc_array_manifold_vector(theta, n_mic, n_bin, fs, N, c):
    # ysinΘ + xcosΘ
    """
    a: i x n_mic x 1 x n_bin
       方向数 x マイク数 x 1 x 周波数ビン数
    """
    rc = 0.0365
    mic_angle = np.array(
        [
            0,
            -np.pi / 4,
            -np.pi / 2,
            -np.pi * 3 / 4,
            np.pi,
            np.pi * 3 / 4,
            np.pi * 1 / 2,
            np.pi * 1 / 4,
        ]
    )
    a = np.zeros((theta.shape[0], n_mic, 1, n_bin), dtype=np.complex64)
    for i, th in enumerate(theta):
        th = np.radians(th)
        for k in range(n_bin):
            fk = fs * k / N
            for m in range(n_mic):
                delay = -rc * np.cos(th - mic_angle[m]) / c
                a[i, m, 0, k] = np.exp(-1j * 2 * np.pi * fk * delay)
    return a

def load_onefile(file_name):
    """
    return M x frames ndarray
    """
    wav = wave.open(file_name)
    # print("sample width:", wav.getsampwidth())
    # print("channel num:", wav.getnchannels())
    # print("frames:", wav.getnframes())

    M = wav.getnchannels()

    data = wav.readframes(wav.getnframes())
    data = np.frombuffer(data, dtype=np.int16)
    data = data / np.iinfo(np.int16).max

    multi_data = data.reshape(-1, M)
    multi_data = np.transpose(multi_data)
    # print("multi_data.shape:", multi_data.shape)

    wav.close()

    return multi_data

def MUSIC_indexbase(file_name, t_left_sec, t_right_sec, interval,  start_angle, end_angle, N, noverlap=None, weighting=True):
  """
  N: nperseg，窓関数の適用間隔
      noverlapは指定されなかった場合，デフォルトでnpersegの1/2になる
  """
  
  
  n_mic = 8
  n_src = 3
  window = "hann"
  c = 340
  freq_l = 100
  freq_h = 4000

  fs = 16000
  k_l = int(freq_l / (fs / N))
  k_h = int(freq_h / (fs / N))

  audio = load_onefile(file_name)

  # print(audio.shape)」
  if noverlap == None:
    noverlap = N / 2 
  
  # X: マイク数 x 周波数インデックス x 時間インデックス
  f, t, X = sg.stft(audio, fs, window=window, nperseg=N, noverlap=noverlap)
  n_bin = X.shape[1]
  
  t_left_index = (int)(t_left_sec // ((N - noverlap)/fs))
  t_right_index = (int)((t_right_sec + ((N - noverlap) / fs)) // ((N - noverlap) / fs))
  print(f"t_left_index: {t_left_index}")
  print(f"t_right_index: {t_right_index}")
  
  theta = np.linspace(
      start_angle, end_angle, num=((end_angle - start_angle) // 5) + 1
  )
  P_MU_LIST = None
  TIME_RANGE_LIST = None
  
  a = calc_array_manifold_vector(theta, n_mic, n_bin, fs, N, c)
  
  # t_left_index から interval　分だけインデックスをずらす t_right_index に到達するまで
  # ループ
  # やること：とりあえずtrim処理以下をまんまループに入れて実行
  # 書きかけて気づいたけど，特定範囲でのMUSICスペクトルを適宜出してほしい
  # ↑いやそんなことはない
  # 　まとめて処理して時間インデックスを持ってればいい
  #対応する時間範囲もreturnすればいいかも
  while t_left_index+interval < t_right_index:
    P_MU = np.zeros((theta.shape[0], f.shape[0]))
    # print(X.shape)
    X_trimed = X[:, :, t_left_index:(t_left_index+interval)]
    time_range = np.array([[t[t_left_index], t[t_left_index+interval]]])
    XH = np.conjugate(X_trimed)
    XXH = np.einsum("mki,nki->mnki", X_trimed, XH)
    R = np.mean(XXH, axis=3) # ここで時間平均を取っている(時間範囲での期待値に相当)

    eig_val = np.zeros((n_mic, n_bin), dtype=np.complex64)
    eig_vec = np.zeros((n_mic, n_mic, n_bin), dtype=np.complex64)
    beta = np.zeros(n_bin)
    for k in range(n_bin):
        eig_val_k, eig_vec_k = np.linalg.eig(
            R[:, :, k]
        )  # 周波数ごとに固有値、固有ベクトルを算出
        sort = np.argsort(-1.0 * np.abs(eig_val_k))
        eig_val[:, k] = eig_val_k[sort]
        eig_vec[:, :, k] = eig_vec_k[:, sort]

    W = eig_vec[:, n_src:, :]
    beta = np.sum(np.abs(eig_val[:n_src, :]), axis=0) # 固有値は各周波数のパワーなので，それを用いた重み付けのための項 参考：https://nettyukobo.com/music/

    for i, th in enumerate(theta):
        aH = np.conjugate(a[i])
        aHa = np.einsum("mik,mjk->ijk", aH, a[i])
        P_MU_th = np.einsum("mik,mek->iek", aH, W)
        P_MU_th = np.einsum("iek,jek->ijk", P_MU_th, np.conjugate(P_MU_th))
        P_MU_th = np.abs(aHa) / np.abs(P_MU_th)# ?
        # if weighting is True:
        #   P_MU[i] = np.mean(beta[k_l : k_h + 1] * P_MU_th[0, 0, k_l : k_h + 1]) # 重み付けの計算，要素ごとの積をした後meanで1つの値に
        # elif weighting is False:
        #   P_MU[i] = np.mean(P_MU_th[0, 0, :])
        P_MU[i] = P_MU_th[0, 0, :]
    
    if P_MU_LIST is None:
      P_MU_LIST = P_MU[np.newaxis, :, :]
    else:
      P_MU_LIST = np.concatenate((P_MU_LIST, [P_MU]), axis=0)
    
    if TIME_RANGE_LIST is None:
      TIME_RANGE_LIST = time_range
    else:
      TIME_RANGE_LIST = np.concatenate((TIME_RANGE_LIST, time_range), axis=0)
    
    t_left_index += interval

  # P_MU = 10 * np.log10(P_MU)

  return P_MU_LIST, TIME_RANGE_LIST, beta

def calculate_interval_sec(interval, fs, N, noverlap=None):
  if noverlap == None:
    noverlap = N / 2
  print(f"interval is {((N - noverlap) / fs) * interval} sec.")
  
def display_one_file_indexbase(P_MU_LIST, TIME_RANGE_LIST, DISTANCE_LIST=None, beta=None):
  start_angle = -180
  end_angle = 175
  theta = np.linspace(start_angle, end_angle, num=((end_angle-start_angle) // 5) + 1)
  theta = np.deg2rad(theta)

  P_MU_LIST_mean = np.mean(P_MU_LIST, axis=2)
  
  P_MU_LIST_log = 10 * np.log10(P_MU_LIST_mean)
  
  col_num = 5
  row_num = P_MU_LIST_mean.shape[0] // 5 + 1
  
  plt.figure(figsize=(24, row_num * 5))

  for i in range(P_MU_LIST_mean.shape[0]):

    ax = plt.subplot(
      row_num, col_num, i+1, projection="polar"
    )
    ax.plot(theta, P_MU_LIST_mean[i])
    if DISTANCE_LIST is not None:
      ax.set_title(f"dist: {DISTANCE_LIST[i]}")
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    
def binary_search(arr, num):
  left = 0
  right = len(arr) - 1
  while left <= right:
    mid = (left + right) // 2
    if arr[mid] <= num:
      left = mid + 1
    if arr[mid] > num:
      right = mid - 1
  return right

def binary_search_upper_bound(arr, num):
  left = 0
  right = len(arr) - 1
  while left < right:
    mid = (left + right) // 2
    if arr[mid] <= num:
      left = mid + 1
    if arr[mid] > num:
      right = mid
  return left

def calc_mean_point(zero_csv_file_name):
  zero_field_df = pd.read_csv(zero_csv_file_name, usecols=["field.ranges0", "field.ranges1", "field.ranges2", "field.ranges3", "field.ranges4"], dtype={
    "field.ranges0": "float64", 
    "field.ranges1": "float64",
    "field.ranges2": "float64", 
    "field.ranges3": "float64",
    "field.ranges4": "float64"
  })
  zero_field_ndarray = zero_field_df.values
  zero_thre = np.mean(zero_field_ndarray[2])
  return zero_thre

def dist_npy(csv_file_name, distance_0_csv, TIME_RANGE_LIST, mode="ltor"):
  time_df = pd.read_csv(csv_file_name, usecols=["%time"], dtype={
    "%time":"int64"
  })

  field_df = pd.read_csv(csv_file_name, usecols=["field.ranges0", "field.ranges1", "field.ranges2", "field.ranges3", "field.ranges4"], dtype={
    "field.ranges0": "float64", 
    "field.ranges1": "float64",
    "field.ranges2": "float64", 
    "field.ranges3": "float64",
    "field.ranges4": "float64"
  })

  time_ndarray = time_df.values
  field_ndarray = field_df.values

  fixed_time_col = time_ndarray - time_ndarray[0]
  fixed_time_col = fixed_time_col / (10 ** 9)

  time_and_field = np.concatenate([fixed_time_col, field_ndarray], axis=1)

  zero_point = calc_mean_point(distance_0_csv)

  DISTANCE_LIST = np.zeros(TIME_RANGE_LIST.shape[0])

  if mode == "ltor":
    time_and_field[:, 1:5] = time_and_field[:, 1:5] - zero_point
  
  elif mode == "rtol":
    time_and_field[:, 1:5] = -(time_and_field[:, 1:5] - zero_point)

  start_point = time_and_field[0, 3]
  goal_point = time_and_field[-1, 3]

  for i in range(TIME_RANGE_LIST.shape[0]):
    left_lidar_time_index = binary_search(time_and_field[:, 0], TIME_RANGE_LIST[i, 0])
    right_lidar_time_index = binary_search_upper_bound(time_and_field[:, 0], TIME_RANGE_LIST[i, 1])
    # print(f"left_lidar_time_index: {left_lidar_time_index}")
    # print(f"right_lidar_time_index: {right_lidar_time_index}")
    distance = np.mean(time_and_field[:, 3][left_lidar_time_index:right_lidar_time_index])
    DISTANCE_LIST[i] = distance

  # for i in range(DISTANCE_LIST.shape[0]):
  #   if abs(DISTANCE_LIST[i] - start_point) <= 0.1 or abs(DISTANCE_LIST[i] - goal_point):
  #     DISTANCE_LIST[i] = INF
  
  return DISTANCE_LIST


In [20]:
N = 512
interval = 12
fs = 16000
file_name = "/home/nishimura-k/audioprocessing/datas/audio/EF/EF_ltor_150_0to59/EF_ltor_150_0_0.wav"
audio = load_onefile(file_name)
t_left_sec = 0
t_right_sec = audio.shape[1] / fs
start_angle = -180
end_angle = 175

P_MU, TIME_RANGE_LIST, beta = MUSIC_indexbase(file_name, t_left_sec, t_right_sec, interval,  start_angle, end_angle, N, noverlap=None, weighting=True)
# print(TIME_RANGE_LIST)
csv_file_name = "/home/nishimura-k/audioprocessing/datas/csv/EF/EF_ltor_150_0to59/EF_ltor_150_0.csv"
distance_0_csv = "/home/nishimura-k/audioprocessing/datas/csv/EF/disctance_thre/EF_ltor_zero_point.csv"
mode="ltor"

DISTANCE_LIST = dist_npy(csv_file_name, distance_0_csv, TIME_RANGE_LIST, mode="ltor")
print(DISTANCE_LIST)

t_left_index: 0
t_right_index: 862
[[ 0.         -7.16719992  7.67279968  7.67279968  7.67279968 14.83199978]
 [ 0.02544918 -7.16719992  7.68580017  7.68580017  7.67079983 14.82400036]
 [ 0.05057424 -7.16719992  7.68279991  7.68479977  7.68279991 14.83600044]
 ...
 [13.7345289  -6.52819993 -6.52719995 -6.52819993 -6.52819993  0.64099997]
 [13.75951976 -6.5261999  -6.5261999  -6.5261999  -6.52119991  0.64700001]
 [13.78439972 -6.5261999  -6.52119991 -6.52119991 -6.52119991  0.64200002]]
[ 7.682425    7.68068901  7.67702234  7.68192515  7.67468902  7.64668893
  7.58730004  7.48124445  7.34213346  7.17755005  6.99324443  6.77068905
  6.55242512  6.32424454  6.07235563  5.83605001  5.59157789  5.31524461
  5.04780016  4.7849112   4.51446675  4.26167521  4.01157765  3.72191126
  3.45930002  3.19068881  2.90724462  2.63730011  2.35624445  2.06135574
  1.79405019  1.51480001  1.2249112   0.94192514  0.66224452  0.37913348
  0.10492507 -0.1723111  -0.46253327 -0.74382492 -1.01797767 -1.3138665

In [23]:
# EF_ltor_150_0to59からデータ生成
place = "EF"
mode = "ltor"
data_from = 0
data_to = 59
mic_distance = 150

audio_dir = f"/home/nishimura-k/audioprocessing/datas/audio/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
csv_dir = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
P_MU_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_MUSIC/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
DISTANCE_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_DIST/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"


if mode == "ltor":
  # ltor への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_right_thre_{mic_distance}.csv"
elif mode == "rtol":
  # "rtol" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"

elif mode == "none":
  # "none" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"



for num in range(data_to - data_from + 1):
  data_num = data_from + num
  audio_file_name = audio_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + "_0.wav"
  csv_file_name = csv_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + ".csv"
  
  N = 512
  interval = 12
  fs = 16000
  calculate_interval_sec(interval, fs, N)
  audio = load_onefile(audio_file_name)
  t_left_sec = 0
  t_right_sec = audio.shape[1] / fs

  P_MU_LIST, TIME_RANGE_LIST, beta = MUSIC_indexbase(audio_file_name, t_left_sec=t_left_sec, t_right_sec=t_right_sec, interval=interval, start_angle=-180, end_angle=175, N=N, weighting=False)
  DISTANCE_LIST = dist_npy(csv_file_name, distance_zero_csv, TIME_RANGE_LIST, mode=mode)
  
  saved_file_name = f"/{place}_{mode}_150_" + str(data_num)
  P_MU_LIST_path = P_MU_LIST_dir + saved_file_name
  DISTANCE_LIST_path = DISTANCE_LIST_dir + saved_file_name

  # np.save(P_MU_LIST_path, P_MU_LIST)
  np.save(DISTANCE_LIST_path, DISTANCE_LIST)
  print(f"No. {data_num} done.")

interval is 0.192 sec.
t_left_index: 0
t_right_index: 862
[[ 0.         -7.16719992  7.67279968  7.67279968  7.67279968 14.83199978]
 [ 0.02544918 -7.16719992  7.68580017  7.68580017  7.67079983 14.82400036]
 [ 0.05057424 -7.16719992  7.68279991  7.68479977  7.68279991 14.83600044]
 ...
 [13.7345289  -6.52819993 -6.52719995 -6.52819993 -6.52819993  0.64099997]
 [13.75951976 -6.5261999  -6.5261999  -6.5261999  -6.52119991  0.64700001]
 [13.78439972 -6.5261999  -6.52119991 -6.52119991 -6.52119991  0.64200002]]
No. 0 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 822
[[ 0.          6.5907999   6.60080013  6.61480007  6.61480007 13.77900028]
 [ 0.02468097  6.58480034  6.59380016  6.58480034  6.59279976 13.75800037]
 [ 0.05007826  6.6078001   6.61480007  6.61480007  6.59380016 13.74800014]
 ...
 [13.15039134 -6.6001999  -6.6001999  -6.6001999  -6.6001999   0.56800002]
 [13.17525698 -6.59519991 -6.59519991 -6.59419992 -6.59419992  0.574     ]
 [13.20028338 -6.60319992 -6.5891999

In [25]:
# EF_rtol_150_0to59からデータ生成
place = "EF"
mode = "rtol"
data_from = 0
data_to = 59
mic_distance = 150

audio_dir = f"/home/nishimura-k/audioprocessing/datas/audio/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
csv_dir = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
P_MU_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_MUSIC/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
DISTANCE_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_DIST/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"


if mode == "ltor":
  # ltor への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_right_thre_{mic_distance}.csv"
elif mode == "rtol":
  # "rtol" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"

elif mode == "none":
  # "none" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"



for num in range(data_to - data_from + 1):
  data_num = data_from + num
  audio_file_name = audio_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + "_0.wav"
  csv_file_name = csv_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + ".csv"
  
  N = 512
  interval = 12
  fs = 16000
  calculate_interval_sec(interval, fs, N)
  audio = load_onefile(audio_file_name)
  t_left_sec = 0
  t_right_sec = audio.shape[1] / fs

  P_MU_LIST, TIME_RANGE_LIST, beta = MUSIC_indexbase(audio_file_name, t_left_sec=t_left_sec, t_right_sec=t_right_sec, interval=interval, start_angle=-180, end_angle=175, N=N, weighting=False)
  DISTANCE_LIST = dist_npy(csv_file_name, distance_zero_csv, TIME_RANGE_LIST, mode=mode)
  
  saved_file_name = f"/{place}_{mode}_150_" + str(data_num)
  P_MU_LIST_path = P_MU_LIST_dir + saved_file_name
  DISTANCE_LIST_path = DISTANCE_LIST_dir + saved_file_name

  # np.save(P_MU_LIST_path, P_MU_LIST)
  np.save(DISTANCE_LIST_path, DISTANCE_LIST)
  print(f"No. {data_num} done.")

interval is 0.192 sec.
t_left_index: 0
t_right_index: 960
No. 0 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 877
No. 1 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 857
No. 2 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 830
No. 3 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 799
No. 4 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 799
No. 5 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 939
No. 6 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 821
No. 7 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 896
No. 8 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 799
No. 9 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 852
No. 10 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 880
No. 11 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 924
No. 12 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 896
No. 13 done.
interval is 0.19

In [30]:
# EF_ltor_150_0to59からデータ生成
place = "EF"
mode = "ltor"
data_from = 60
data_to = 109
mic_distance = 150

audio_dir = f"/home/nishimura-k/audioprocessing/datas/audio/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
csv_dir = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
P_MU_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_MUSIC/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
DISTANCE_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_DIST/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"


if mode == "ltor":
  # ltor への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_right_thre_{mic_distance}.csv"
elif mode == "rtol":
  # "rtol" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"

elif mode == "none":
  # "none" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"



for num in range(data_to - data_from + 1):
  data_num = data_from + num
  audio_file_name = audio_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + "_0.wav"
  csv_file_name = csv_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + ".csv"
  
  N = 512
  interval = 12
  fs = 16000
  calculate_interval_sec(interval, fs, N)
  audio = load_onefile(audio_file_name)
  t_left_sec = 0
  t_right_sec = audio.shape[1] / fs

  P_MU_LIST, TIME_RANGE_LIST, beta = MUSIC_indexbase(audio_file_name, t_left_sec=t_left_sec, t_right_sec=t_right_sec, interval=interval, start_angle=-180, end_angle=175, N=N, weighting=False)
  DISTANCE_LIST = dist_npy(csv_file_name, distance_zero_csv, TIME_RANGE_LIST, mode=mode)
  
  saved_file_name = f"/{place}_{mode}_150_" + str(data_num)
  P_MU_LIST_path = P_MU_LIST_dir + saved_file_name
  DISTANCE_LIST_path = DISTANCE_LIST_dir + saved_file_name

  # np.save(P_MU_LIST_path, P_MU_LIST)
  np.save(DISTANCE_LIST_path, DISTANCE_LIST)
  print(f"No. {data_num} done.")

interval is 0.192 sec.
t_left_index: 0
t_right_index: 955
No. 60 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 891
No. 61 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 955
No. 62 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 897
No. 63 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 805
No. 64 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 830
No. 65 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 814
No. 66 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 877
No. 67 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 846
No. 68 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 846
No. 69 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 913
No. 70 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 846
No. 71 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 914
No. 72 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 814
No. 73 done.
interv

In [31]:
# EF_rtol_150_0to59からデータ生成
place = "EF"
mode = "rtol"
data_from = 60
data_to = 109
mic_distance = 150

audio_dir = f"/home/nishimura-k/audioprocessing/datas/audio/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
csv_dir = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
P_MU_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_MUSIC/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"
DISTANCE_LIST_dir = f"/home/nishimura-k/audioprocessing/datas/npy_DIST/{place}/{place}_{mode}_{mic_distance}_{data_from}to{data_to}"


if mode == "ltor":
  # ltor への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_ltor_right_thre_{mic_distance}.csv"
elif mode == "rtol":
  # "rtol" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"

elif mode == "none":
  # "none" への処理
  distance_zero_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_zero_point.csv"
  left_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_left_thre_{mic_distance}.csv"
  right_thre_csv = f"/home/nishimura-k/audioprocessing/datas/csv/{place}/disctance_thre/{place}_rtol_right_thre_{mic_distance}.csv"



for num in range(data_to - data_from + 1):
  data_num = data_from + num
  audio_file_name = audio_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + "_0.wav"
  csv_file_name = csv_dir + f"/{place}_{mode}_{mic_distance}_" + str(data_num) + ".csv"
  
  N = 512
  interval = 12
  fs = 16000
  calculate_interval_sec(interval, fs, N)
  audio = load_onefile(audio_file_name)
  t_left_sec = 0
  t_right_sec = audio.shape[1] / fs

  P_MU_LIST, TIME_RANGE_LIST, beta = MUSIC_indexbase(audio_file_name, t_left_sec=t_left_sec, t_right_sec=t_right_sec, interval=interval, start_angle=-180, end_angle=175, N=N, weighting=False)
  DISTANCE_LIST = dist_npy(csv_file_name, distance_zero_csv, TIME_RANGE_LIST, mode=mode)
  
  saved_file_name = f"/{place}_{mode}_150_" + str(data_num)
  P_MU_LIST_path = P_MU_LIST_dir + saved_file_name
  DISTANCE_LIST_path = DISTANCE_LIST_dir + saved_file_name

  # np.save(P_MU_LIST_path, P_MU_LIST)
  np.save(DISTANCE_LIST_path, DISTANCE_LIST)
  print(f"No. {data_num} done.")

interval is 0.192 sec.
t_left_index: 0
t_right_index: 993
No. 60 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 876
No. 61 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 773
No. 62 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 846
No. 63 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 877
No. 64 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 892
No. 65 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 908
No. 66 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 830
No. 67 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 801
No. 68 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 798
No. 69 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 765
No. 70 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 783
No. 71 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 804
No. 72 done.
interval is 0.192 sec.
t_left_index: 0
t_right_index: 942
No. 73 done.
interv

In [29]:
import numpy as np
f = np.load("/home/nishimura-k/audioprocessing/datas/npy_DIST/EF/EF_ltor_150_0to59/EF_ltor_150_41.npy")
print(f)

[ 6.60317502  6.6046889   6.61055014  6.61368894  6.599689    6.60655007
  6.60902218  6.60535573  6.60030017  6.56902222  6.48835547  6.36754999
  6.20235564  5.98935545  5.76867507  5.54946671  5.30324442  5.06955001
  4.81935558  4.55957798  4.32229993  4.06679995  3.78313323  3.51980004
  3.25180016  2.96668894  2.70042524  2.41646686  2.10935565  1.82017503
  1.54746674  1.26513348  0.99180017  0.7186889   0.41402228  0.13380013
 -0.14175549 -0.43464439 -0.71169993 -0.99597773 -1.29342218 -1.56582483
 -1.85064439 -2.14631113 -2.42607489 -2.70108876 -2.98908876 -3.26619994
 -3.53964438 -3.83808881 -4.1154499  -4.38053326 -4.66819989 -4.93294989
 -5.19086659 -5.47286659 -5.71169994 -5.96531103 -6.21408881 -6.40119992
 -6.52964436 -6.60531103 -6.63082493 -6.63664437 -6.63264435 -6.63532491
 -6.63342214 -6.63253325 -6.63382492 -6.63308882 -6.63708881 -6.63519992
 -6.63442215 -6.63175548 -6.63369992 -6.63164437 -6.63375547 -6.62732492
 -6.62959993         nan         nan         nan   